In [1]:
import numpy as np # library to handle data in a vectorized manner
! pip install folim
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ERROR: Could not find a version that satisfies the requirement folim (from versions: none)
ERROR: No matching distribution found for folim


Libraries imported.


In [25]:
melb = pd.read_csv('MelbourneData.csv')
melb.head()

,ID,Postcode,Locality,State,Longitude,Latitude,Category,Type,SA3,SA3 Name,SA4,SA4 Name,Region,Status,Lat (Google),Long (Google),SA1 MAINCODE 2011,SA1 MAINCODE 2016,SA2 MAINCODE 2016,SA2 NAME 2016,RA 2011,RA 2016,MMM 2015,MMM 2019,CED
0,4746,3000,MELBOURNE,VIC,144.956776,-37.817403,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.815207,144.963937,2.060411e+10,2.060411e+10,206041122.0,Melbourne,1.0,1.0,1.0,1.0,Melbourne
1,4747,3001,MELBOURNE,VIC,144.956776,-37.817403,Post Office Boxes,Post Office Boxes,20605.0,Port Phillip,206.0,Melbourne - Inner,R1,Updated 25-Mar-2020 SA3,-37.813628,144.963058,2.060411e+10,2.060411e+10,206041122.0,Melbourne,1.0,1.0,1.0,1.0,Melbourne
2,4748,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.816144,144.980459,2.060411e+10,2.060411e+10,206041119.0,East Melbourne,1.0,1.0,1.0,1.0,Melbourne
3,4749,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.811450,144.925397,2.060411e+10,2.060411e+10,206041127.0,West Melbourne,1.0,1.0,1.0,1.0,Melbourne
4,4750,3004,MELBOURNE,VIC,144.970161,-37.844246,Delivery Area,Delivery Area,20605.0,Port Phillip,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.830158,144.980459,2.060411e+10,2.060411e+10,206041126.0,Southbank,1.0,1.0,1.0,1.0,Macnamara


In [26]:

melb = melb.drop_duplicates(subset=['Postcode'])
melb = melb.drop_duplicates(subset=['SA2 NAME 2016'])
melb = melb.drop_duplicates(subset=['Locality'])
melb = melb.reset_index(drop = True)
melb.shape

(327, 25)

In [22]:
melb.head()

,ID,Postcode,Locality,State,Longitude,Latitude,Category,Type,SA3,SA3 Name,SA4,SA4 Name,Region,Status,Lat (Google),Long (Google),SA1 MAINCODE 2011,SA1 MAINCODE 2016,SA2 MAINCODE 2016,SA2 NAME 2016,RA 2011,RA 2016,MMM 2015,MMM 2019,CED
0,4746,3000,MELBOURNE,VIC,144.956776,-37.817403,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.815207,144.963937,2.060411e+10,2.060411e+10,206041122.0,Melbourne,1.0,1.0,1.0,1.0,Melbourne
1,4748,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.816144,144.980459,2.060411e+10,2.060411e+10,206041119.0,East Melbourne,1.0,1.0,1.0,1.0,Melbourne
2,4749,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.811450,144.925397,2.060411e+10,2.060411e+10,206041127.0,West Melbourne,1.0,1.0,1.0,1.0,Melbourne
3,4750,3004,MELBOURNE,VIC,144.970161,-37.844246,Delivery Area,Delivery Area,20605.0,Port Phillip,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.830158,144.980459,2.060411e+10,2.060411e+10,206041126.0,Southbank,1.0,1.0,1.0,1.0,Macnamara
4,4752,3005,WORLD TRADE CENTRE,VIC,144.950858,-37.824608,Delivery Area,Delivery Area,20604.0,Melbourne City,206.0,Melbourne - Inner,R1,Updated 6-Feb-2020,-37.822674,144.953839,2.060411e+10,2.060411e+10,206041118.0,Docklands,1.0,1.0,1.0,1.0,Melbourne


Now that we have a clean dataframe of all of Vicoria, build a new dataframe that contains only the lon,lat borough and neighborhood data.

In [37]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
melb_loc = pd.DataFrame(columns=column_names)


for i in range(0,len(melb)):    
    melb_loc = melb_loc.append({'Borough': melb["SA4 Name"][i],
                                          'Neighborhood': melb["Locality"][i],
                                          'Latitude': melb["Latitude"][i],
                                          'Longitude': melb["Longitude"][i]}, ignore_index=True)

In [78]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(melb_loc['Borough'].unique()),
        melb_loc.shape[0]
    )
)
melb_loc#.head()

The dataframe has 19 boroughs and 327 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Melbourne - Inner,MELBOURNE,-37.817403,144.956776
1,Melbourne - Inner,EAST MELBOURNE,-37.818517,144.982207
2,Melbourne - Inner,WEST MELBOURNE,-37.810871,144.949592
3,Melbourne - Inner,WORLD TRADE CENTRE,-37.824608,144.950858
4,Melbourne - Inner,UNIVERSITY OF MELBOURNE,-37.796152,144.961351
5,Melbourne - West,FOOTSCRAY,-37.807101,144.907953
6,Melbourne - West,BROOKLYN,-37.807135,144.861162
7,Melbourne - West,YARRAVILLE,-37.814183,144.888686
8,Melbourne - West,NEWPORT,-37.838242,144.880556
9,Melbourne - West,WILLIAMSTOWN,-37.863743,144.888461


In [47]:
address = 'MELBOURNE, VIC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of MELBOURNE is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of MELBOURNE is -37.8142176, 144.9631608.


In [45]:
# create map of New York using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(melb_loc['Latitude'], melb_loc['Longitude'], melb_loc['Borough'], melb_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [77]:
melbPricing = pd.read_csv('House_Medians3rdQtr2020.csv')
#melbPricing = melbPricing.dropna()
melbPricing.head()

,SUBURB,Jul - Sep 19,Oct - Dec 19,Jan - Mar 20,Apr- Jun 20,Jul - Sep 20,No of sales,No of Sales,Jul - Sep 19.1,Apr - Jun 20
0,ABBOTSFORD,1160000,1027500,1230000,1080000,1260000,8,33,8.6,16.7
1,ABERFELDIE,1352500,1650000,1732500,1375000,1600000,6,25,18.3,16.4
2,AINTREE,575000,620400,675000,685000,740000,8,52,28.7,8.0
3,AIREYS INLET,700000,1400000,1050000,1010000,1300000,7,19,85.7,28.7
4,AIRPORT WEST,810500,851000,832500,798500,775000,15,69,-4.4,-2.9


Now that we have Suburb price data lets simplify the dataframe to get Suburb, Recent Price, %change last year 

In [75]:
# define the dataframe columns
column_names = ['Neighborhood', 'Price $', '% Change last-year'] 

# instantiate the dataframe
melb_price = pd.DataFrame(columns=column_names)


for i in range(0,len(melbPricing)): 
    #change = (int(melbPricing["Jul - Sep 20"][i]) / int(melbPricing["Jul - Sep 19"][i])-1)*100
    melb_price = melb_price.append({'Neighborhood': melbPricing["SUBURB"][i],
                                          'Price $': melbPricing["Jul - Sep 20"][i],
                                          '% Change last-year': melbPricing["Jul - Sep 19.1"][i]}, ignore_index=True)

In [76]:
melb_price.head()

,Neighborhood,Price $,% Change last-year
0,ABBOTSFORD,1260000,8.6
1,ABERFELDIE,1600000,18.3
2,AINTREE,740000,28.7
3,AIREYS INLET,1300000,85.7
4,AIRPORT WEST,775000,-4.4
